In [2]:
# 1. Importing the Libraries
import os
#os.chdir("../input/assignment")
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
    from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch import optim
import numpy as np
from  data import ChristmasImages

%matplotlib inline

In [3]:
# HyperParameters
epochs = 3
learning_rate = 0.0001
batch_size = 32

In [4]:
# 2. loading the data
path = os. getcwd()

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((128,128)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# datasets
train_set = ChristmasImages(path, transform = transform)
test_set = ChristmasImages(path, training = False, transform = transform)

# dataloaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size, shuffle = True)

# Setting up Cuda device
'''device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)'''

"device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\nprint(device)"

In [ ]:
# 3. Import network
from model import Network
mymodel = Network()
#mymodel.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mymodel.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.9)

In [ ]:
# 4. Train the network

# Defining accuracy function
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)    
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

# iterating over the epochs
for e in range(epochs):
    running_loss = 0
    output = None
    acc = 0
    # Iterating over the batches
    for i,data in enumerate(train_loader, 0):        
        # Preparing data
        images, labels = data
        #images, labels = images.to(device), labels.to(device)
        
        # Set gradient to zeros
        optimizer.zero_grad()       
        
        #prediction, then optimizing the parameters
        mymodel.train()
        output = mymodel(images)      
        
        loss = criterion(output, labels)        
        loss.backward() # backward propagation
        optimizer.step() #performs a single optimization step(updates the parameters)
        
        #accumulating loss and accuracy
        running_loss += loss.item()
        acc += accuracy(output, labels)
        scheduler.step()
        
        #printing intermediate loss
        if i % 5 == 4:
            print('[%d, %5d] loss: %.3f, accuracy:%4d ' % (e + 1, i + 1, running_loss / 5, (acc*100 / 5)))
            running_loss = 0.0
            acc = 0
print("Model is trained." )
        

In [ ]:
mymodel.save_model()

In [5]:
#loading the model
from model import Network

model = Network()
model.load_state_dict(torch.load('model'))
model.eval()
#model.to(device)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Network(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): 

In [6]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)    
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

running_loss_test = 0
acc = 0
for images, labels in test_loader:                     
    #images, labels = images.to(device), labels.to(device)
    output = (model(images))  
    _, preds = torch.max(output, dim=1) 
    acc += accuracy(output,labels)             



print( "Test accuracy: %d" % (acc*100/(len(test_loader))) )

testing loss:  0.0
Test accuracy: 83
